# Ngày 2: Resilient Distributed Datasets

### 1. Khái niệm RDD

*Resilient distributed dataset (RDDs)* là tập dữ liệu phân tán và có khả năng phục hồi

a. Phân tán Distributed

![Distributed](image_1/distribute.png)

**Giải thích**: Hình trên giả sử có một file dữ liệu được lưu ở HDFS. Spark có 3 worker thực hiện song song nhiệm vụ sử lí, các dữ liệu là các block. Sau đó Spark thực hiệnmột transformation load dữ liệu lên **Memory** tạothành 3 partition để xử lí dữ liệu, và việc transformation load dữ liệu lên Memory tạo nên 1 RDD
<div style="text-align: center; color:green;">
    RDD1 = Load file to Memory
</div>
- Lưu ý: Các transformation thực chất chỉ để spark ghi nhớ chứ chưa hoàn toàn được thực thi. Đến khi có một
action thì spark sẽ thực thi với chiến lược thông minh nhất


b. Có khả năng phục hồi (Resilent)

![Resilient](image_1/resilient.png)

**Giải thích**: Hình trên tạo nên các RDD. Thì giả sử RDD3 bị lỗi, lúc này spark hoàn toàn biết được cha của 
RDD3 là RDD2, vì vậy nó sẽ từ RDD2 đó tạo lại RDD3 chứ không chạy lại từ đầu.

Note: Vì chạy trên memory và chiến lược Lazy Transformation nên spark chạy cực kì nhanh.



### 2. Các đặc điểm của RDD

- **Immutable (Bất biến)**: Một khi đã tạo ra RDD, bạn không thể thay đổi nó. Thay vào đó, bạn tạo RDD mới từ RDD gốc thông qua các phép biến đổi (transformations).

- **Distributed (Phân tán)**: RDD tự động chia nhỏ dữ liệu thành các phân vùng và phân phối chúng qua các nodes trong cụm Spark.

- **Fault-tolerant (Chịu lỗi)**: Nếu một phần của RDD bị mất, Spark có thể xây dựng lại nó từ các RDD trước đó nhờ lineage (dòng dõi) của RDD.

### 3. Tạo RDD:

Có hai các tạo: Dùng **parralleize** hoặc **External**s data

a. Parallelize: dùng hàm _parallelize()_ để tạo RDD từ danh sách hoặc mảng
```python
    from pyspark import SparkContext

    sc = SparkContext("local", "RDD Example")

    # Tạo RDD từ collection
    data = [1, 2, 3, 4, 5]
    rdd = sc.parallelize(data)
```

b. External data: Tạo từ một tập dữ liệu có sẵn (textFile(), .......)
```python
    from pyspark import SparkContext
    
    sc = SparkContext("local", "RDD Example")
    # Tạo RDD từ tập tin văn bản
    file_rdd = sc.textFile("hdfs://path/to/file.txt")
```



### 4. Các loại RDD

a. <u>__textFile__</u>: Đọc dữ liệu từ file văn bản, mỗi <span style="color:#B22222;">dòng</span> là một phần tử của RDD.
```python
      text_rdd = sc.textFile("hdfs://path/to/textfile.txt")
```


b.<u> __wholeTextFiles__:</u> Đọc toàn bộ nội dung file vào RDD dưới dạng <span style="color:#B22222;">(fileName, content). </span> 
```python
      # Đọc toàn bộ nội dung của các file trong thư mục
      files_rdd = sc.wholeTextFiles("hdfs://path/to/directory")

      # Hiển thị tên file và nội dung của chúng
      for file_name, content in files_rdd.take(3):
      print(f"File: {file_name}")
      print(f"Content: {content}")
```


c. <u>__sequenceFile__:</u> Đọc file dạng <span style="color:#B22222;">key-value.</span>
```python
      # Đọc dữ liệu từ một file Sequence (key-value)
      sequence_rdd = sc.sequenceFile("hdfs://path/to/sequencefile"
                                    , keyClass="org.apache.hadoop.io.Text"
                                    , valueClass="org.apache.hadoop.io.IntWritable")

      # Hiển thị key-value của 5 cặp đầu tiên
      for key, value in sequence_rdd.take(5):
      print(f"Key: {key}, Value: {value}")

```


d. <u>__HadoopRDD__:</u> Sử dụng Hadoop InputFormat để đọc dữ liệu từ HDFS hoặc các nguồn khác.Sử dụng Hadoop InputFormat để đọc dữ liệu từ HDFS hoặc các nguồn khác.
```python
      # Đọc dữ liệu từ HDFS sử dụng Hadoop InputFormat
      hadoop_rdd = sc.hadoopFile("hdfs://path/to/input"
                              , inputFormatClass="org.apache.hadoop.mapred.TextInputFormat"
                              , keyClass="org.apache.hadoop.io.LongWritable"
                              , valueClass="org.apache.hadoop.io.Text")

      # Hiển thị key-value của 5 cặp đầu tiên
      for key, value in hadoop_rdd.take(5):
      print(f"Key: {key}, Value: {value}")


```

### 5. Các Transformation cơ bản 

Trong bài này, ta sẽ tìm hiểu về 3 transformation cơ bản là: *Filter*, *Map* và *FlatMap*

1. **Filter**: Đúng như tên gọi của nó, dùng để lọc dữ liệu, kết quả là một RDD mới
```python
    # filter: Lọc các số lẻ
    rdd_filtered = rdd.filter(lambda x: x % 2 != 0)
```


2. **Map**: Áp dụng một hàm lên từng phần tử của RDD, kết quả là một RDD mới.
```python
    # map: Tính bình phương của từng số
    rdd_squared = rdd.map(lambda x: x * x)
```

3. **FlatMap**
```python
    # flatMap: Tách câu thành từ
    lines = sc.parallelize(["Hello world", "Apache Spark"])
    words = lines.flatMap(lambda line: line.split(" "))
```


Thắc mắc: Sự khác biệt giữa Map và flatMap là gì ?
<details>
  <summary>Đáp án:</summary>
    <b>1. map(): </b>
    <p>- Mô tả: map() biến đổi từng phần tử của RDD (hoặc tập hợp) bằng cách áp dụng một hàm lên từng phần tử đó. Kết quả là một RDD mới với số lượng phần tử không thay đổi (mỗi phần tử đầu vào tạo ra đúng một phần tử đầu ra). </p>
    <p>- Đầu vào/Đầu ra: Một phần tử đầu vào -> Một phần tử đầu ra. </p>
    <b>2. flatMap(): </b>
        <p>- Mô tả: flatMap() cũng áp dụng một hàm lên từng phần tử của RDD, nhưng khác ở chỗ nó cho phép mỗi phần tử đầu vào ánh xạ thành một hoặc nhiều phần tử đầu ra. Sau đó, các phần tử đầu ra này được "dẹp phẳng" (flattened) thành một danh sách duy nhất.. </p>
        <p>- Đầu vào/Đầu ra: Một phần tử đầu vào -> Một hoặc nhiều phần tử đầu ra (có thể là 0). </p>

    
</details>




## Bài tập ngày 2

**Bài 1: Tạo RDD từ một tập tin văn bản lớn và đếm số lần xuất hiện của từng từ.**



In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("WordCount_2") \
    .getOrCreate()

24/10/23 13:02:50 WARN Utils: Your hostname, ubunchuu-Test resolves to a loopback address: 127.0.1.1; using 172.18.0.1 instead (on interface br-753e92851dac)
24/10/23 13:02:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/23 13:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/10/23 13:03:05 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
sc = spark.sparkContext

In [4]:
rdd1 = sc.textFile("material_1/wordcount.txt")

In [5]:
rdd1.collect()

['Hello my name is Nghia',
 'I am ten years old',
 'I live in Ho Chin Minh City now',
 'I study at University of Science, in Ho Chi Minh city, Viet Nam',
 'Vinh is my friend. He also live in Ho Chi Minh']

In [6]:
rdd2 = rdd1.flatMap(lambda x: x.split(" "))

In [7]:
rdd3 = rdd2.map(lambda word :(word, 1))

In [8]:
rdd4 = rdd3.reduceByKey(lambda x, y: x+y )

In [9]:
rdd4.collect()

[('Hello', 1),
 ('name', 1),
 ('is', 2),
 ('am', 1),
 ('ten', 1),
 ('years', 1),
 ('live', 2),
 ('in', 3),
 ('Chin', 1),
 ('now', 1),
 ('at', 1),
 ('University', 1),
 ('of', 1),
 ('Science,', 1),
 ('Chi', 2),
 ('Viet', 1),
 ('Nam', 1),
 ('friend.', 1),
 ('my', 2),
 ('Nghia', 1),
 ('I', 3),
 ('old', 1),
 ('Ho', 3),
 ('Minh', 3),
 ('City', 1),
 ('study', 1),
 ('city,', 1),
 ('Vinh', 1),
 ('He', 1),
 ('also', 1)]

**Bài 2: Lọc ra các dòng dữ liệu thỏa mãn một điều kiện cụ thể từ RDD.**

In [10]:
# Ví dụ ta lấy dữ liệu từ bài 1 trên, hãy in ra các dòng dữ liệu có chứa string "Chi Minh"

rdd_filter = rdd1.filter(lambda line: "Chi Minh" in line)

In [11]:
rdd_filter.collect()

['I study at University of Science, in Ho Chi Minh city, Viet Nam',
 'Vinh is my friend. He also live in Ho Chi Minh']

**Bài 3: Áp dụng một hàm tùy chỉnh lên từng phần tử của RDD.**



In [12]:
# Ta lại lấy dữ liệu từ ví dụ trên, với kết quả rdd2 được làm phảng, mỗi phần tử ta tạo thành một list gồm nó và kí tự spark

# Cach 1 : Xu dung lambda

rdd_addString = rdd2.map(lambda x: [x,"Spark"])

In [13]:
rdd_addString.collect()

[['Hello', 'Spark'],
 ['my', 'Spark'],
 ['name', 'Spark'],
 ['is', 'Spark'],
 ['Nghia', 'Spark'],
 ['I', 'Spark'],
 ['am', 'Spark'],
 ['ten', 'Spark'],
 ['years', 'Spark'],
 ['old', 'Spark'],
 ['I', 'Spark'],
 ['live', 'Spark'],
 ['in', 'Spark'],
 ['Ho', 'Spark'],
 ['Chin', 'Spark'],
 ['Minh', 'Spark'],
 ['City', 'Spark'],
 ['now', 'Spark'],
 ['I', 'Spark'],
 ['study', 'Spark'],
 ['at', 'Spark'],
 ['University', 'Spark'],
 ['of', 'Spark'],
 ['Science,', 'Spark'],
 ['in', 'Spark'],
 ['Ho', 'Spark'],
 ['Chi', 'Spark'],
 ['Minh', 'Spark'],
 ['city,', 'Spark'],
 ['Viet', 'Spark'],
 ['Nam', 'Spark'],
 ['Vinh', 'Spark'],
 ['is', 'Spark'],
 ['my', 'Spark'],
 ['friend.', 'Spark'],
 ['He', 'Spark'],
 ['also', 'Spark'],
 ['live', 'Spark'],
 ['in', 'Spark'],
 ['Ho', 'Spark'],
 ['Chi', 'Spark'],
 ['Minh', 'Spark']]

In [14]:
# Cach 2: Xu dung ham do minh viet

def addString_2(x):
    return [x,"Spark"]

rdd_add_2 = rdd2.map(addString_2)

rdd_add_2.collect()

[['Hello', 'Spark'],
 ['my', 'Spark'],
 ['name', 'Spark'],
 ['is', 'Spark'],
 ['Nghia', 'Spark'],
 ['I', 'Spark'],
 ['am', 'Spark'],
 ['ten', 'Spark'],
 ['years', 'Spark'],
 ['old', 'Spark'],
 ['I', 'Spark'],
 ['live', 'Spark'],
 ['in', 'Spark'],
 ['Ho', 'Spark'],
 ['Chin', 'Spark'],
 ['Minh', 'Spark'],
 ['City', 'Spark'],
 ['now', 'Spark'],
 ['I', 'Spark'],
 ['study', 'Spark'],
 ['at', 'Spark'],
 ['University', 'Spark'],
 ['of', 'Spark'],
 ['Science,', 'Spark'],
 ['in', 'Spark'],
 ['Ho', 'Spark'],
 ['Chi', 'Spark'],
 ['Minh', 'Spark'],
 ['city,', 'Spark'],
 ['Viet', 'Spark'],
 ['Nam', 'Spark'],
 ['Vinh', 'Spark'],
 ['is', 'Spark'],
 ['my', 'Spark'],
 ['friend.', 'Spark'],
 ['He', 'Spark'],
 ['also', 'Spark'],
 ['live', 'Spark'],
 ['in', 'Spark'],
 ['Ho', 'Spark'],
 ['Chi', 'Spark'],
 ['Minh', 'Spark']]

In [15]:
spark.stop()